# Libraries

In [16]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.preprocessing import scale
from collections import deque
import random
import numpy as np
import time
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default = 'jupyterlab'
# import plotly.offline as po

I have used dataset from KAGGLE

# Data Importing and Preprocessing

In [2]:
data = pd.read_csv(r'D:\Data Analytics for Business\Keras learning\NSE Coal India\BAJAJFINSV__EQ__NSE__NSE__MINUTE.csv')
data.head().T

,0,1,2,3,4
timestamp,2017-01-02 09:15:00+05:30,2017-01-02 09:16:00+05:30,2017-01-02 09:17:00+05:30,2017-01-02 09:18:00+05:30,2017-01-02 09:19:00+05:30
open,2898.0,2901.4,2900.85,2871.65,2885.0
high,2901.45,2913.0,2900.85,2885.0,2895.0
low,2897.0,2895.1,2864.2,2867.0,2884.95
close,2901.4,2899.25,2868.3,2885.0,2890.0
volume,347.0,1419.0,642.0,712.0,637.0


In [3]:
seq_length = 80                     # how many sequence should we use to predict
future_predict_period =2            # based on sequence data we are trying to predict for next 2 periods

In [4]:
data.describe()

,open,high,low,close,volume
count,370408.000000,370408.000000,370408.000000,370408.000000,370408.000000
mean,6197.359667,6200.971033,6193.591429,6197.316345,976.804375
std,1615.999775,1616.535168,1615.381194,1615.985123,2107.229357
min,2871.650000,2875.000000,2864.200000,2868.300000,0.000000
25%,5069.500000,5072.050000,5065.550000,5069.350000,123.000000
50%,5964.000000,5968.200000,5960.000000,5964.000000,352.000000
75%,7169.062500,7173.000000,7164.312500,7169.012500,982.000000
max,9925.550000,9950.000000,9925.000000,9925.100000,133328.000000


**Timestamp is a string and has hours and minutes so we need to remove them,then convert it to a datetime and put in the index.**

**We only need close and volume so need to remove everything apart from these two.**

In [5]:
data['timestamp'] =[ n[:10] for n in data['timestamp']]
for c in data.columns:
    if c != 'timestamp':
        data[c]  = data[c].astype('float')
data['date'] =pd.to_datetime(data['timestamp'],format='%Y-%m-%d')
data = data.set_index(data['date'])

data = data.drop(['open','high','low','date','timestamp'],1)


<ipython-input-5-b6a93e7b59f0>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(['open','high','low','date','timestamp'],1)


In [6]:
data.head()

,close,volume
date,,
2017-01-02,2901.40,347.0
2017-01-02,2899.25,1419.0
2017-01-02,2868.30,642.0
2017-01-02,2885.00,712.0
2017-01-02,2890.00,637.0


**created a future parameter that is basically a 3 step ahead future of close.**

**I have created it to generate a target that will return 1 if future price will rise and 0 when future price will fall**

In [7]:
data['future'] = data['close'].shift(-future_predict_period)

In [8]:
def target(current,future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [9]:
data['target'] = list( map(target,data['close'],data['future']) )

**sorted the time index and kept the last 5% of the data as the validation set**

In [10]:
time = sorted(data.index.values)
time_5pct = time[-int(0.05*len(time))]
validation_data = data[(data.index>=time_5pct)]
training_data = data[(data.index< time_5pct)]

**Preprocessing function**
first we need to drop future because otherwise our model will not be able to learn.
Then i have scaled all the columns.pct_change was generating infinite values so replaced them with na and dropped them.
I have used sklearn scaler to scale the column.

Created a sequential data of max length 60 using deque and appending data of 60 observations as a 1 observation to the sequencial data.
Then balanced the no of 0 and 1 in the data and returned the x and y


In [11]:
def preprocessing(data):
    data = data.drop('future',1)
    for col in data.columns:
        if col !='target':
            data[col] = data[col].pct_change()
            data.dropna(inplace=True)
            data[col].replace(np.inf,np.nan,inplace=True)
            data[col].dropna(inplace=True)
            data[col] = scale(data[col].values)

    data.dropna(inplace=True)    

    sequencial_data = []
    prev_days = deque(maxlen=seq_length)

    for i in data.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days)==seq_length:
            sequencial_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequencial_data)

    buys = []
    sells = []
    for o,v in sequencial_data:
        if v==0:
            sells.append([o,v])
        elif v==1:
            buys.append([o,v])
    
    lower = min(len(sells),len(buys))
    buys = buys[:lower]
    sells = sells[:lower]
    print(len(buys),len(sells))    
    sequencial_data = buys+sells
    random.shuffle(sequencial_data)
    x = []
    y= []

    for seq,tar in sequencial_data:
        x.append(seq)
        y.append(tar)

    return np.array(x),y


In [12]:
x_train,y_train = preprocessing(training_data)
x_test,y_test = preprocessing(validation_data)


<ipython-input-11-a822a9edc9a7>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('future',1)


168155 168155


<ipython-input-11-a822a9edc9a7>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('future',1)


9149 9149


In [13]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

# Model Creation and Evaluation

**Created a LSTM model followed by a dense layer.**
**Using dropout of 0.2**

In [14]:
def give_model(neurons=128,opt='adam',dense=64):
    model = Sequential()
    model.add(LSTM(neurons,input_shape=(x_train.shape[1:]),return_sequences=True,activation='tanh'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(neurons,input_shape=(x_train.shape[1:]),return_sequences=True,activation='tanh'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(neurons,input_shape=(x_train.shape[1:]),return_sequences=True,activation='tanh'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(neurons,input_shape=(x_train.shape[1:]),activation='tanh'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(dense,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(optimizer=opt,loss=SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])
    return model

**I have used Adam as an optimizer with initial lr of 0.001 and decay of 1e-6. Used ModelCheckpoint to checkpointing the data and tensorboard to visualise the model**

In [17]:
name = f'{seq_length}_{future_predict_period}_{int(time.time())}'

In [18]:
# opt= Adam(learning_rate=0.01)
tensorboard = TensorBoard(log_dir=f'logs/{name}')
chkpt = ModelCheckpoint(filepath= r'D:\Data Analytics for Business\Keras learning\NSE Coal India\best_model.hdf5',monitor='val_accuracy')

In [19]:
cv = KerasRegressor(build_fn=give_model)
params = {'opt':['adam','sgd'],'batch_size':[64,128],'neurons':[64,128,256],'dense':[32,64,128]}

In [ ]:
randomized_cvv = RandomizedSearchCV(cv,params)
randomized_cvv.fit(x_train,y_train,epochs=1)

In [22]:
randomized_cvv.best_params_

{'opt': 'adam', 'neurons': 128, 'dense': 128, 'batch_size': 64}

In [19]:
model = give_model(neurons=128,dense=128,opt='adam')

In [20]:
hist = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=64,epochs=200,callbacks=[tensorboard,chkpt])

Epoch 1/200
5255/5255 [==============================] - 203s 36ms/step - loss: 0.6964 - accuracy: 0.5237 - val_loss: 0.6904 - val_accuracy: 0.5278
Epoch 2/200
5255/5255 [==============================] - 185s 35ms/step - loss: 0.6904 - accuracy: 0.5354 - val_loss: 0.6914 - val_accuracy: 0.5322
Epoch 3/200
5255/5255 [==============================] - 187s 36ms/step - loss: 0.6893 - accuracy: 0.5428 - val_loss: 0.6899 - val_accuracy: 0.5393
Epoch 4/200
5255/5255 [==============================] - 184s 35ms/step - loss: 0.6889 - accuracy: 0.5420 - val_loss: 0.6894 - val_accuracy: 0.5344
Epoch 5/200
5255/5255 [==============================] - 183s 35ms/step - loss: 0.6880 - accuracy: 0.5464 - val_loss: 0.6885 - val_accuracy: 0.5394
Epoch 6/200
5255/5255 [==============================] - 185s 35ms/step - loss: 0.6877 - accuracy: 0.5478 - val_loss: 0.6886 - val_accuracy: 0.5345
Epoch 7/200
5255/5255 [==============================] - 183s 35ms/step - loss: 0.6875 - accuracy: 0.5479 - val_

KeyboardInterrupt: 

In [61]:

joblib.dump(randomized_cvv, 'randomized_cvv.pkl')

['randomized_cvv.pkl']

In [38]:
ypred_train = model.predict(x_train)

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.plot(np.arange(0,10),hist.history['val_accuracy'],label ='Validation')
plt.plot(np.arange(0,10),hist.history['accuracy'],label ='Train')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(np.arange(0,10),hist.history['val_loss'],label ='Validation')
plt.plot(np.arange(0,10),hist.history['loss'],label ='Train')
plt.tight_layout()
plt.title('Loss')
plt.legend()

In [ ]:
fig = make_subplots(rows=1,cols=2,x_title='Epochs',subplot_titles=('Accuracy','Loss'),horizontal_spacing=0.12 )

fig.add_trace(trace=go.Scatter(x=np.arange(0,10),y=hist.history['val_accuracy'],name='Validation'),row=1,col=1 )
fig.add_trace(trace =go.Scatter(x=np.arange(1,11),y=hist.history['accuracy'],name='Train'),row=1,col=1)

fig.add_trace(trace =go.Scatter(x=np.arange(1,11),y=hist.history['val_loss'],name='Validation'),row=1,col=2)
fig.add_trace(trace =go.Scatter(x=np.arange(1,11),y=hist.history['loss'],name='Train'),row=1,col=2)

fig.update_layout(height=600, width=1200)
fig.update_xaxes(range=(1,10))
fig.show()